In [3]:
import numpy as np
import pandas as pd


In [4]:
cd ml-100k/

C:\Users\young desi\ml-100k


In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 844F-DEC2

 Directory of C:\Users\young desi\ml-100k

12/12/2016  12:44 AM    <DIR>          .
12/12/2016  12:44 AM    <DIR>          ..
07/19/2000  03:09 PM               716 allbut.pl
07/19/2000  03:09 PM               643 mku.sh
01/29/2016  02:26 PM             6,750 README
07/19/2000  03:09 PM         1,979,173 u.data
07/19/2000  03:09 PM               202 u.genre
07/19/2000  03:09 PM                36 u.info
07/19/2000  03:09 PM           236,344 u.item
07/19/2000  03:09 PM               193 u.occupation
07/19/2000  03:09 PM            22,628 u.user
03/08/2001  12:33 PM         1,586,544 u1.base
03/08/2001  12:32 PM           392,629 u1.test
03/08/2001  12:33 PM         1,583,948 u2.base
03/08/2001  12:33 PM           395,225 u2.test
03/08/2001  12:33 PM         1,582,546 u3.base
03/08/2001  12:33 PM           396,627 u3.test
03/08/2001  12:33 PM         1,581,878 u4.base
03/08/2001  12:33 PM           397,295 u4.test
03/08/20

In [5]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]


In [7]:
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    #print row[1],row[2],row[3]
    ratings[row[1]-1, row[2]-1] = row[3]
ratings
def split_data(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings] 
    return train, test
train, test = split_data(ratings)

In [8]:
from sklearn.metrics import pairwise_distances

item_correlation = 1 - pairwise_distances(train.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0.

In [9]:

idx_to_movie = {}
with open('u.item', 'r') as f:
    for line in f.readlines():
        info = line.split('|')
        idx_to_movie[int(info[0])-1] = info[4]
        
def top_movies(similarity, mp, movie_idx, k=6):
    return [mp[i] for i in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

In [10]:
id=1
movies = top_movies(item_correlation, idx_to_movie, id)
print movies

['http://us.imdb.com/M/title-exact?GoldenEye%20(1995)', 'http://us.imdb.com/M/title-exact?Under%20Siege%20(1992)', 'http://us.imdb.com/M/title-exact?Stargate%20(1994)', 'http://us.imdb.com/M/title-exact?True%20Lies%20(1994)', 'http://us.imdb.com/M/title-exact?Top%20Gun%20(1986)', 'http://us.imdb.com/M/title-exact?Cliffhanger%20(1993)']


http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)
http://us.imdb.com/M/title-exact?Independence%20Day%20(1996)
http://us.imdb.com/M/title-exact?Willy%20Wonka%20and%20the%20Chocolate%20Factory%20(1971)
http://us.imdb.com/M/title-exact?Mission:%20Impossible%20(1996)
http://us.imdb.com/M/title-exact?Beauty%20and%20the%20Beast%20(1991)
http://us.imdb.com/M/title-exact?Rock,%20The%20(1996)


In [13]:
data=train
lmbda = 0.1 
k = 20
m, n = data.shape 
n_epochs = 15 
Ivalue = data.copy()
Ivalue[Ivalue > 0] = 1
Ivalue[Ivalue == 0] = 0



P_user = 3 * np.random.rand(k,m) 
Q_item = 3 * np.random.rand(k,n) 
Q_item[0,:] = R[R != 0].mean(axis=0) 
E = np.eye(k)

train_errors = []
test_errors = []

def rmse(Ivalue,data,Q_item,P_user):
    return np.sqrt(np.sum((I * (data - np.dot(P_user.T,Q_item)))**2)/len(data[data > 0]))

iterations=0
while iterations > n_epochs:
    
    # Fix Q and estimate P
    for i, Ii in enumerate(I):
        n_i_rated = np.count_nonzero(Ii) 
        if (n_i_rated == 0): 
            n_i_rated = 1
    
       
        A_value = np.dot(Q_item, np.dot(np.diag(Ii), Q_item.T)) + lmbda * n_i_rated * E
        V_value = np.dot(Q_item, np.dot(np.diag(Ii), data[i].T))
        P[:,i] = np.linalg.solve(A_value,V_value)
        
  
    for j, Ij in enumerate(Ivalue.T):
        nmj = np.count_nonzero(Ij) 
        if (nmj == 0): 
            nmj = 1 
        
        # Least squares solution
        Aj = np.dot(P, np.dot(np.diag(Ij), P.T)) + lmbda * nmj * E
        Vj = np.dot(P, np.dot(np.diag(Ij), R[:,j]))
        Q[:,j] = np.linalg.solve(Aj,Vj)
    
    train_rmse = rmse(Ivalue,data,Q_item,P_user)
    
    train_errors.append(train_rmse)
    iterations=iterations+1
    
    
    
    
print "Training complete"

Algorithm converged


In [22]:
predicted = pd.DataFrame(np.dot(P_user.T,Q_item))
R = pd.DataFrame(test)


         0         1         2         3         4         5         6     \
0    3.896384  3.086417  3.167939  3.634344  2.733475  4.066825  4.110688   
1    3.788634  3.107309  2.558171  3.471509  3.135183  3.894344  3.676837   
2    3.289037  2.405071  2.315854  2.933410  2.784952  2.622783  3.264377   
3    5.044949  4.350879  3.712692  3.959472  4.312310  4.341347  4.795080   
4    3.443269  2.848845  2.162046  3.446844  2.315225  2.744261  3.919435   
5    3.401299  2.583006  2.076503  3.245878  2.536327  3.077460  3.249194   
6    4.105433  3.616949  2.958457  3.894261  3.790847  3.741852  3.787257   
7    3.992770  2.884424  3.187987  3.425561  3.302916  4.222282  3.877351   
8    3.998326  3.629697  2.623267  4.027361  3.675588  4.118348  3.753503   
9    4.112971  3.379775  2.999785  3.706504  3.624602  3.806646  3.958995   
10   3.504672  2.908079  2.487958  3.351493  2.996435  3.187640  3.672673   
11   4.106582  3.614928  3.036270  3.865317  3.482058  3.484478  3.889063   

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rms=0
for i in range(0,910):
    ratings1 = pd.DataFrame(data1=R.loc[i,R.loc[i,:] > 0]).head(n=1600)
    ratings1['Prediction'] = predicted.loc[i,R.loc[i,:] > 0]
    ratings1.columns = ['Actual Rating', 'Predicted']
    rms = rms+sqrt(mean_squared_error(ratings1['Actual'],ratings1['Predicted']))
print rms/910

In [32]:
import pickle
pickle.dump(P , open( "C:\\Users\\young desi\\Documents\\recommender\\person_svd.p", "wb" ) )
pickle.dump(Q , open( "C:\\Users\\young desi\\Documents\\recommender\\item_svd.p", "wb" ) )
pickle.dump(R , open( "C:\\Users\\young desi\\Documents\\recommender\\ratings.p", "wb" ) )

In [6]:
i_cols = ['movie_id', 'movie_title' ,'release_date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movie_base = pd.read_csv('C:\data\ml-100k\u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [8]:
movie_base.head()

,movie_id,movie_title,release_date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
from sklearn.cluster import KMeans

In [7]:
X=np.array(df[['user_id', 'item_id', 'rating']])

In [8]:
X

array([[ 196,  242,    3],
       [ 186,  302,    3],
       [  22,  377,    1],
       ..., 
       [ 276, 1090,    1],
       [  13,  225,    2],
       [  12,  203,    3]], dtype=int64)

In [9]:
kmeans = KMeans(n_clusters=20)
kmeans.fit(X)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

In [11]:
centroids

array([[  181.35445791,  1020.41130786,     3.11556384],
       [  381.90757535,    82.70885569,     3.71845826],
       [  770.13554084,   719.05231788,     3.31611479],
       [   96.2810837 ,   482.79075955,     3.5892598 ],
       [  518.80244664,   267.74869859,     3.57431026],
       [  395.72131747,   715.3021043 ,     3.39762123],
       [  789.42475987,   995.15297047,     3.16542156],
       [  118.37508821,    79.2154199 ,     3.75299929],
       [  745.15433572,  1353.97295147,     2.93317422],
       [  853.96441465,   103.60632628,     3.75691937],
       [  797.52729045,   271.85783904,     3.55332776],
       [  483.34062885,   983.24352651,     3.18249075],
       [  338.95547782,   477.07870886,     3.55430116],
       [  841.10575793,   481.87990599,     3.54007051],
       [  124.24027211,   731.19265306,     3.33823129],
       [  297.12504732,   237.49463722,     3.67810726],
       [   90.88382015,   257.70495569,     3.63554316],
       [  586.5768057 ,   504.2

In [12]:
labels = kmeans.predict(X)

C:\Users\young desi\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [38]:
labels[1300:1490]

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6])

In [13]:
user_movies2=np.array(df.loc[df['user_id']==196 ])

In [209]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def find_user_movies(user_id):
    user_movies2=np.array(df.loc[df['user_id']==user_id ])
    #print user_movies2
    mov=np.array(df.loc[df['user_id']==195 ])
    r1=[]
    r2=[]
    
    for movies in user_movies2:
       # print movies
        for m,K,L,N in mov:
           # print m,K,L,N
            if movies[1]==K:
                 r1.append(movies[2])
                 r2.append(L)
    r2=np.array(r2)
    r1=np.array(r1)
    #print r2,r1
    rms = sqrt(mean_squared_error(r1,r2))
    return rms
    

In [211]:
j=[]
j=np.where(labels==labels[195])
#np.size(j[0])
rms1=0
count=0
for i in j[0]:
    
    rms1=rms1+find_user_movies(i)
    count=count+1
rms1=rms1/count
print count
  

1.51338117493


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [71]:
df1=df.loc[df['user_id'] == j[0][2]]
df2=df1.loc[df['rating'] == 1]
df2

,user_id,item_id,rating,timestamp
3271,6,477,1,883599509
5145,6,476,1,883600175
9602,6,458,1,883599914
25228,6,465,1,883683508
31510,6,259,1,883268375
38333,6,405,1,883600066
43410,6,472,1,883600003


In [72]:
df3=df.loc[df['user_id'] == 195]
df4=df3.loc[df3['item_id'].isin (df2['item_id'])]
df4

,user_id,item_id,rating,timestamp
5293,195,477,2,885110922


NameError: name 'y_actual' is not defined

{0: 1044,
 1: 4568,
 2: 60,
 3: 2333,
 4: 390,
 5: 3853,
 6: 1316,
 7: 184,
 8: 618,
 9: 1077,
 10: 2326,
 11: 4653,
 12: 2017,
 13: 125,
 14: 2581}